# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

# 1. Подготовка данных

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

data = pd.read_csv('/datasets/Churn.csv')
data.info()
print(data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             9091 non-null float64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB
   RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age  \
0          1    15634602  Hargrave          619    France  Female   42   
1          2    15647311      Hill          608     Spain  Female   41   
2          3    15619304      Onio      

- RowNumber - дубликат индекса, удаляем

In [ ]:
data.drop('RowNumber', axis = 1, inplace = True)
data.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


- Tenure - есть пропуски(нужно удалить или заменить

In [ ]:
data['Tenure'] = data['Tenure'].fillna(data['Tenure'].median())# заполним медианой

- Surname, Geography, Gender - object. Нужно перевести в числа, чтобы модель работала.

In [ ]:
#Посмотрим уникальные объекты в Gender
data.Gender.unique()

array(['Female', 'Male'], dtype=object)

In [ ]:
#Заменим на 0 и 1
data.Gender = data.Gender.map({'Female': 0, 'Male':1})
data.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,15634602,Hargrave,619,France,0,42,2.0,0.00,1,1,1,101348.88,1
1,15647311,Hill,608,Spain,0,41,1.0,83807.86,1,0,1,112542.58,0
2,15619304,Onio,502,France,0,42,8.0,159660.80,3,1,0,113931.57,1
3,15701354,Boni,699,France,0,39,1.0,0.00,2,0,0,93826.63,0
4,15737888,Mitchell,850,Spain,0,43,2.0,125510.82,1,1,1,79084.10,0


In [ ]:
# Заполним Surname и Geography числами 
encoder = OrdinalEncoder()
data_ordinal = encoder.fit_transform(data)
data_new = pd.DataFrame(data_ordinal, columns=data.columns)
data_new.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,2736.0,1115.0,228.0,0.0,0.0,24.0,2.0,0.0,0.0,1.0,1.0,5068.0,1.0
1,3258.0,1177.0,217.0,2.0,0.0,23.0,1.0,743.0,0.0,0.0,1.0,5639.0,0.0
2,2104.0,2040.0,111.0,0.0,0.0,24.0,8.0,5793.0,2.0,1.0,0.0,5707.0,1.0
3,5435.0,289.0,308.0,0.0,0.0,21.0,1.0,0.0,1.0,0.0,0.0,4704.0,0.0
4,6899.0,1822.0,459.0,2.0,0.0,25.0,2.0,3696.0,0.0,1.0,1.0,3925.0,0.0


In [ ]:
data_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
CustomerId         10000 non-null float64
Surname            10000 non-null float64
CreditScore        10000 non-null float64
Geography          10000 non-null float64
Gender             10000 non-null float64
Age                10000 non-null float64
Tenure             10000 non-null float64
Balance            10000 non-null float64
NumOfProducts      10000 non-null float64
HasCrCard          10000 non-null float64
IsActiveMember     10000 non-null float64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null float64
dtypes: float64(13)
memory usage: 1015.8 KB


In [ ]:
data_new = data_new.astype({
    'CustomerId'    : 'int32',
    'Surname'       : 'int32',
    'Geography'     : 'int32',
    'Gender'        : 'int32',
    'Age'           : 'int32',
    'Tenure'        : 'int32',
    'NumOfProducts' : 'int32',
    'HasCrCard'     : 'int32',
    'IsActiveMember': 'int32',
    'Exited'        : 'int32'})
data_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
CustomerId         10000 non-null int32
Surname            10000 non-null int32
CreditScore        10000 non-null float64
Geography          10000 non-null int32
Gender             10000 non-null int32
Age                10000 non-null int32
Tenure             10000 non-null int32
Balance            10000 non-null float64
NumOfProducts      10000 non-null int32
HasCrCard          10000 non-null int32
IsActiveMember     10000 non-null int32
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int32
dtypes: float64(3), int32(10)
memory usage: 625.1 KB


## Вывод:

После изучения данных обнаружил:
 -  столбец RowNumber - дубликат индекса, удалил его
 -  столбцы Surname, Geography, Gender - object. Перевел в числа, чтобы модель работала.
 -  чтобы ускорить обработку данных, все столбцы кроме CreditScore, Balance, EstimatedSalary, перевел в int32

# 2. Исследование задачи

- **Разделим датасет на тренировочную, валидационную и тестовую выборки**

In [ ]:
target = data_new['Exited']
features = data_new.drop('Exited', axis=1)

features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)

In [ ]:
# Разделим валидационную на тестовую и валидационную
features_test, features_valid, target_test, target_valid = train_test_split(
    features_valid, target_valid, test_size=0.5, random_state=12345)

<div class="alert alert-block alert-success">
<b>Успех:</b> Разделение данных на тренировочные, валидацию и тест сделано корректно.
    
</div>

- **Проверим модель решающего дерева**

In [ ]:
model = DecisionTreeClassifier(random_state=12345)
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)

print(f1_score(target_valid, predicted_valid))

0.4878957169459963


- **Проверим модель случайных деревьев**

In [ ]:
ran_forest = RandomForestClassifier(random_state=12345, n_estimators=10)
ran_forest.fit(features_train, target_train) 
predicted_valid_forest = ran_forest.predict(features_valid)

print(f1_score(target_valid, predicted_valid_forest))

0.508557457212714


- **Проверим модель логистической регрессии**

In [ ]:
log_reg = LogisticRegression(random_state=12345, solver='liblinear', class_weight='balanced')
log_reg.fit(features_train, target_train)
predicted_valid_regr = log_reg.predict(features_valid)

print("F1:", f1_score(target_valid, predicted_valid_regr))

F1: 0.4927536231884058


- **Лучший скор у случайных деревьев. Подберем для него параметры.**

In [ ]:
# Используем GridSearchCV для поиска оптимальных гиперпараметров
params = [
    {
        'n_estimators': [50, 100, 150],
        'max_leaf_nodes':[50,100,150]
    },
    {
        'bootstrap': [False],
        'n_estimators': [50, 100,150],
    },
]
ran_forest = RandomForestClassifier()
grid_search = GridSearchCV(
    ran_forest, param_grid = params, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(features_train, target_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             iid

In [ ]:
grid_search.best_params_

{'max_leaf_nodes': 150, 'n_estimators': 50}

In [ ]:
#Используем лучшие параметры для модели
ran_forest = RandomForestClassifier(
    random_state=12345,
    n_estimators=50,
    max_leaf_nodes=150
)
ran_forest.fit(features_train, target_train) 
predicted_valid_forest = ran_forest.predict(features_valid)

print('VALID       f1      =', f1_score(target_valid, predicted_valid_forest))
print('VALID accuracy      =', accuracy_score(target_valid, predicted_valid_forest))
print('VALID roc_auc_score =', roc_auc_score(target_valid, predicted_valid_forest))

VALID       f1      = 0.5037783375314862
VALID accuracy      = 0.8424
VALID roc_auc_score = 0.6710423433176952


## Вывод:

 - Разделил датасет на тренировочную, валидационную и тестовую выборки
 - Проверил работу моделей: тестового дерева, случайных деревьев и логистической регрессии. 
 - Выбрал модель с лучшие показания f1-меры - Модель случайных деревьев.
 - Для выбранной модели методом GridSearchCV подобрал параметры.
 - Проверил значения f1, accuracy и roc_auc_score на валидационных данных.

# 3. Борьба с дисбалансом

In [ ]:
target_train.value_counts()

0    4804
1    1196
Name: Exited, dtype: int64

- **Видим количество значений 0 целевого признака в 4 раза больше, чем 1.**
**Применим метод upsampling/downsampling для балансировки признаков**

In [ ]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]   
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    #Создаем новый датафрейм с признаками
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    
    #Создаем новый датафрейм с целевыми значениями
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    #Перемешиваем датафрейм
    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_train_u, target_train_u = upsample(features_train, target_train, 4)
target_train_u.value_counts()

1    6008
0    5998
Name: Exited, dtype: int64

In [ ]:
#Проверим модель после upsample
ran_forest = RandomForestClassifier(
    random_state=12345,
    n_estimators=50,
    max_leaf_nodes=150
)
ran_forest.fit(features_train_u, target_train_u) 
predicted_valid_forest = ran_forest.predict(features_valid)
print('VALID       f1      =', f1_score(target_valid, predicted_valid_forest))
print('VALID accuracy      =', accuracy_score(target_valid, predicted_valid_forest))
print('VALID roc_auc_score =', roc_auc_score(target_valid, predicted_valid_forest))

VALID       f1      = 0.6175548589341693
VALID accuracy      = 0.8048
VALID roc_auc_score = 0.7757315349452664


In [ ]:
def downsampling(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]   
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    #Создаем новый датафрейм с признаками
    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    
    #Создаем новый датафрейм с целевыми значениями
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    #Перемешиваем датафрейм
    features_downsampled, target_downsampled = shuffle(features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

for i in np.arange(0.04, 0.41, 0.04):
    features_downsampled, target_downsampled = downsampling(features_train, target_train, i)
    model = RandomForestClassifier(random_state=12345)
    model.fit(features_downsampled, target_downsampled)
    predictions = model.predict(features_valid)
    print("F1_Score: {: >5.3f}  |  Fractions : {: >5.3f}".format(f1_score(target_valid, predictions), i))

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


F1_Score: 0.412  |  Fractions : 0.040
F1_Score: 0.464  |  Fractions : 0.080
F1_Score: 0.503  |  Fractions : 0.120
F1_Score: 0.551  |  Fractions : 0.160


/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


F1_Score: 0.568  |  Fractions : 0.200
F1_Score: 0.570  |  Fractions : 0.240
F1_Score: 0.568  |  Fractions : 0.280


/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


F1_Score: 0.565  |  Fractions : 0.320
F1_Score: 0.592  |  Fractions : 0.360
F1_Score: 0.557  |  Fractions : 0.400


In [ ]:
#Проверим модель после downsample
ran_forest_down = RandomForestClassifier(
    random_state=12345,
    n_estimators=50,
    max_leaf_nodes=150
)
features_downsampled, target_downsampled = downsampling(features_train, target_train, 0.36)
ran_forest_down.fit(features_downsampled, target_downsampled) 
predicted_valid_forest = ran_forest_down.predict(features_valid)
print('VALID       f1      =', f1_score(target_valid, predicted_valid_forest))
print('VALID accuracy      =', accuracy_score(target_valid, predicted_valid_forest))
print('VALID roc_auc_score =', roc_auc_score(target_valid, predicted_valid_forest))

VALID       f1      = 0.6352941176470587
VALID accuracy      = 0.8264
VALID roc_auc_score = 0.7789193131240226


## Вывод

 - Обнаружил дисбаланс данных. Значений целевого признака 0 в 4 раза больше, чем 1.
 - Использовал upsampling/downsampling для балансировки признаков.
 - Выбрал downsampling, т.к. у него выше показания метрик.
 - Проверил на валидационных данных.

# 4. Тестирование модели

 - **Протестируем модель на тестовой выборке.**

In [ ]:
features_downsampled, target_downsampled = downsampling(features_train, target_train, 0.36)
ran_forest_d = RandomForestClassifier(random_state=1234,
    n_estimators=50,
    max_leaf_nodes=150)
ran_forest_d.fit(features_downsampled, target_downsampled)
predictions_test = ran_forest_d.predict(features_test)

print('TEST       f1      =', f1_score(target_test, predictions_test))
print('TEST accuracy      =', accuracy_score(target_test, predictions_test))
print('TEST roc_auc_score =', roc_auc_score(target_test, predictions_test))

TEST       f1      = 0.6156521739130436
TEST accuracy      = 0.8232
TEST roc_auc_score = 0.7681128433899246


# Вывод: 

 - Проверил полученную модель на тестовой выборке.
 - Значение метрики f1 удовлетворяет требованию(выше 0.59)
 - Дополнительной измерил accuracy и roc_auc_score.
 - Высокий AUC ROC и высокий f1 означает, что классификатор в настоящее время выполняет достойную работу, и для многих других значений порога он будет делать то же самое.